
Okay, it's time for us to start working with real data! This project will require you to product two notebooks: one that processes data and dumps data to a pickle file, a second notebook that loads the pickle file and applies regression models to predict song popularity. Proposed due date: Friday October 26th, 5pm.

The following link is a fairly famous unprocessed data set which collected listening habits of approximately 1000 users: http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/lastfm-1K.html

The data set is large (about 650 MB compressed), data is formatted one entry per line, tab-separated. The first task is to process this data to recover some song statistics. In particular, we would like to create (and save, using pickle), a pandas dataframe that contains:

the number of times a song has been played
percentage of the plays that came from users who self-identified as male
average age of the listener
percentage of plays that came from a user who played the song exactly once
percentage of plays that came from a user who played the song at least five times

If there any missing values, decide how you will deal with them, and summarize your approach.

In this same pre-processing notebook, explore the data by generating some descriptive statistics. After each code block, you should have summary statements, or code blocks should print out summary statements that are useful to a reader.

Complete this notebook by exporting the generated dataframe using pickle for posterity, but don't commit your pickled file to the repository. Sometimes, it is more convenient to process the data locally rather than transmitting large processed data files.

# Import, Preprocess, Merge, and Pickle 

In [1]:
# Import the relevant packages
import pandas as pd
import numpy as np
import pickle
%matplotlib inline

## Songs

In [2]:
# Read the data with the song info
songs_names=['UserID','Timestamp','ArtistID','Artist','TrackID','Track']
df_songs = pd.read_csv("C:\\Users\\rpena\Desktop\\lastfm-dataset-1K.tar\\lastfm-dataset-1K\\userid-timestamp-artid-artname-traid-traname.tsv",sep='\t',names=songs_names)

In [3]:
df_songs.shape

(835873, 6)

In [4]:
df_songs.head(5)

,UserID,Timestamp,ArtistID,Artist,TrackID,Track
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


The columns containing "Timestamp", "ArtistID", and "TrackID" are not needed for the analysis we are running. These columns were removed from the dataset to make it more interpretable. In any model these will have 100% correlation with Artist Name and TrackName, respectively and do not add information gained by modeling. The code chunk below removes them and confirms that they were removed without affecting the rows.

In [5]:
# Create a subset of df_songs using a copy of the original. Leaves the original dataframe intact
songs = df_songs[["UserID","Track"]].copy()

In [6]:
songs.shape

(835873, 2)

In [7]:
songs.head(5)

,UserID,Track
0,user_000001,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,Composition 0919 (Live_2009_4_15)
2,user_000001,Mc2 (Live_2009_4_15)
3,user_000001,Hibari (Live_2009_4_15)
4,user_000001,Mc1 (Live_2009_4_15)


The output above shows that the irrelavant columns were dropped and that all of the rows remain.

### Descriptive Statistics - Songs

In [8]:
songs.UserID.describe()

count          835873
unique             33
top       user_000012
freq            75876
Name: UserID, dtype: object

There are 33 unique userIDs with the most frequent user being user_000012.

In [9]:
songs.Track.describe()

count        835873
unique       116682
top       Heartless
freq           2121
Name: Track, dtype: object

There are 11682 unique Tracks in this dataset with Heartless being the most frequent track played.

### Missing Values - Songs

In [10]:
songs.isnull().sum() / len(songs)

UserID    0.0
Track     0.0
dtype: float64

This code shows that there are 0 missing values in rows across all three columns of this dataset.  

No imputation or removal is required.

## Users

The dataset for the User information also contains columns that are not useful for our analysis.   

Below, these columns are dropped and summary statistics are run on the remaining columns.

In [11]:
# Read the data with the Users info
df_users=pd.read_csv("C:\\Users\\rpena\Desktop\\lastfm-dataset-1K.tar\\lastfm-dataset-1K\\userid-profile.tsv",sep='\t')
df_users.head(5)

,#id,gender,age,country,registered
0,user_000001,m,NaN,Japan,"Aug 13, 2006"
1,user_000002,f,NaN,Peru,"Feb 24, 2006"
2,user_000003,m,22.0,United States,"Oct 30, 2005"
3,user_000004,f,NaN,NaN,"Apr 26, 2006"
4,user_000005,m,NaN,Bulgaria,"Jun 29, 2006"


In [12]:
df_users.shape

(992, 5)

In [13]:
# Create a subset of df_users using a copy of the original. Leaves the original dataframe intact
users = df_users[["#id","gender","age"]].copy()

In [14]:
# Rename columns
users=users.rename(columns={'#id':'UserID','gender':'Gender','age':'Age'})

In [15]:
# Remove unnecessary columns
users = users[["UserID","Gender","Age"]]
users.head(5)

,UserID,Gender,Age
0,user_000001,m,NaN
1,user_000002,f,NaN
2,user_000003,m,22.0
3,user_000004,f,NaN
4,user_000005,m,NaN


In [16]:
# Check the shape of the new dataframe
users.shape

(992, 3)

### Descriptive Statistics - Users 

In [17]:
users.UserID.describe()

count             992
unique            992
top       user_000178
freq                1
Name: UserID, dtype: object

Descriptive statistics of UserID show that there are over 992 unique users in this dataset.

In [18]:
users.Gender.describe()

count     884
unique      2
top         m
freq      502
Name: Gender, dtype: object

Descriptive statistics of Gender show that there are two unique categories with the most frequent gender being Male at 502 of 884. 

This does not equal the number of observations (992) determined using shape which indicates missing values that will need to be handled later.

In [19]:
users.Age.describe()

count    286.000000
mean      25.367133
std        8.314952
min        3.000000
25%       21.000000
50%       23.000000
75%       28.000000
max      103.000000
Name: Age, dtype: float64

Descriptive statistics of Age show that average age of Users is 25. 

The most interesting results for this set are the Minimum and Maximum values of Age, 3 and 103. 

These are most likely input errors (intentional or otherwise) and will need to be handled. 

The Count of these values is also significantly less than expected at 286 of the 992 observations. 

This indicates a very large number of missing values.

In [20]:
users.loc[(users['Age']< 10) | (users['Age'] > 90),['Age']]= np.nan

In [21]:
users.Age.describe()

count    282.000000
mean      25.312057
std        6.638221
min       15.000000
25%       21.000000
50%       23.500000
75%       28.000000
max       75.000000
Name: Age, dtype: float64

The code chunk above changes any Age values that are less than 14 and greater than 90 into null values. 

This brought the Minimum Age up to 18 and the Maximum Age to 75 as well as lowering the standard deviation.

Next, missing values will be handled.

### Missing Values - Users

In [22]:
users.isnull().sum() / len(users)

UserID    0.000000
Gender    0.108871
Age       0.715726
dtype: float64

The number of missing values in Age column render this predictor almost useless. 

Typically, this predictor would not be useful even after imputation; it would be dropped. 

For the purpose of this project I will impute the mean of Ages in place of the missing values.

In [23]:
users.Age.fillna(users.Age.mean(),inplace=True);

In [24]:
users.isnull().sum() / len(users)

UserID    0.000000
Gender    0.108871
Age       0.000000
dtype: float64

In [25]:
#Change dtype of Age from float to int now that imputation is complete.
users.Age = users.Age.astype(int)

In [26]:
users.Age.describe()

count    992.00000
mean      25.08871
std        3.53763
min       15.00000
25%       25.00000
50%       25.00000
75%       25.00000
max       75.00000
Name: Age, dtype: float64

Impute the mode for the missing values in Gender.

The missing values in Gender will be filled with 'm' as it is more likely to be a male based on exploratory analysis.

In [27]:
users.Gender.value_counts()

m    502
f    382
Name: Gender, dtype: int64

In [28]:
users.Gender.fillna('m',inplace=True,)

In [29]:
users.isnull().sum() / len(users)

UserID    0.0
Gender    0.0
Age       0.0
dtype: float64

## Merging the two DataFrames
Inner joins return those records that have matching values in both tables.

This join was done on the relation of "UserID" between the two dataframes. 

In [30]:
data = pd.merge(users, songs, on="UserID", how='inner')
data.head(20)

,UserID,Gender,Age,Track
0,user_000001,m,25,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,m,25,Composition 0919 (Live_2009_4_15)
2,user_000001,m,25,Mc2 (Live_2009_4_15)
3,user_000001,m,25,Hibari (Live_2009_4_15)
4,user_000001,m,25,Mc1 (Live_2009_4_15)
5,user_000001,m,25,To Stanford (Live_2009_4_15)
6,user_000001,m,25,Improvisation (Live_2009_4_15)
7,user_000001,m,25,Glacier (Live_2009_4_15)
8,user_000001,m,25,Parolibre (Live_2009_4_15)
9,user_000001,m,25,Bibo No Aozora (Live_2009_4_15)


In [31]:
data.shape

(835873, 4)

In [32]:
data.Track.describe()

count        835873
unique       116682
top       Heartless
freq           2121
Name: Track, dtype: object

In [33]:
data.UserID.describe()

count          835873
unique             33
top       user_000012
freq            75876
Name: UserID, dtype: object

In [34]:
data.Track.value_counts().head(10)

Heartless                                          2121
See You In My Nightmares                           2069
Say You Will                                       2069
Love Lockdown                                      2062
Welcome To Heartbreak (Feat. Kid Cudi)             2059
Coldest Winter                                     2052
Paranoid (Feat. Mr. Hudson)                        2051
Amazing (Feat. Young Jeezy)                        2049
Pinocchio Story (Freestyle Live From Singapore)    2041
Bad News                                           2028
Name: Track, dtype: int64

### Number of times a song has been played

The code below creates a pivot table of the merged dataframes to subset and aggregate total plays per song.

In [35]:
plays = data.pivot_table(index=["Track"],aggfunc='count')
plays.head()

,Age,Gender,UserID
Track,,,
! (Foreword),8,8,8
"# 173 - The Stills, Novillero, Fred Eaglesmith",1,1,1
"# 174 - The Organ, Sloan, Jill Barber",2,2,2
#1,65,65,65
#1 Crush,7,7,7


In [36]:
plays=plays.drop(['UserID','Gender'],axis=1)

In [37]:
plays=plays.rename(columns={'Age':'Plays'})
plays.head(5)

,Plays
Track,
! (Foreword),8
"# 173 - The Stills, Novillero, Fred Eaglesmith",1
"# 174 - The Organ, Sloan, Jill Barber",2
#1,65
#1 Crush,7


### Percentage of the plays that came from users who self-identified as male  

Similar to the last chunk of code, a pivot table was created with the aggregate count of genders taken.

In [38]:
data.Gender.value_counts()

m    546882
f    288991
Name: Gender, dtype: int64

In [39]:
genders_pt = data.pivot_table(index=['Track'],columns="Gender",aggfunc='count')

In [40]:
genders_pt.head(5)

Age       UserID      
Gender                                             f     m      f     m
Track                                                                  
! (Foreword)                                     NaN   8.0    NaN   8.0
# 173 - The Stills, Novillero, Fred Eaglesmith   NaN   1.0    NaN   1.0
# 174 - The Organ, Sloan, Jill Barber            1.0   1.0    1.0   1.0
#1                                              29.0  36.0   29.0  36.0
#1 Crush                                         1.0   6.0    1.0   6.0

In [41]:
genders_pt=genders_pt.drop('UserID',axis=1)
genders_pt.fillna(0,inplace=True,)
genders_pt.head(20)

Age      
Gender                                             f     m
Track                                                     
! (Foreword)                                     0.0   8.0
# 173 - The Stills, Novillero, Fred Eaglesmith   0.0   1.0
# 174 - The Organ, Sloan, Jill Barber            1.0   1.0
#1                                              29.0  36.0
#1 Crush                                         1.0   6.0
#1 Crush (Original Version)                     10.0   0.0
#1 Hit Jam                                       0.0   2.0
#1 Target                                        0.0   1.0
#102 - Saskatoon Secret                          0.0   1.0
#105 - The Canadian Dictionary Special           0.0   1.0
#107 - Prom Date                                 0.0   1.0
#111 - Turning Green                             0.0   1.0
#112 - Polaris Music Prize Shortlist 2007        0.0   1.0
#141 – Toast To The East Coast                   0.0   1.0
#142 – The Make-Out Podcast                      0.0   1.0
#143 – Sleepless Maritime Nights                 0.0   1.0
#144 – Here And Now With Hayden                  0.0   1.0
#146 - Sxsw Survival Guide                       0.0   1.0
#150 - Your Weekly Canadian Mixtape              0.0   1.0
#151 – Winnersleep                               0.0   1.0

This function calculates the percentage of male listeners.

In [42]:
def pct_male(row):
    total= (row['Age']['m'] + row['Age']['f'])
    return ((row['Age']['m']/total)*100)

The above function is applied to the pivot table and saved in the dataframe of features of interest.

In [43]:
plays["Pct Males"] = round(genders_pt.apply(pct_male, axis=1),2)

In [44]:
plays.head(5)

,Plays,Pct Males
Track,,
! (Foreword),8,100.00
"# 173 - The Stills, Novillero, Fred Eaglesmith",1,100.00
"# 174 - The Organ, Sloan, Jill Barber",2,50.00
#1,65,55.38
#1 Crush,7,85.71


### Average age of the listener  

Another pivot table is used to aggregate the mean of Age.

In [45]:
age = data.pivot_table(index=['Track'],aggfunc="mean")

In [46]:
age.head(5)

,Age
Track,
! (Foreword),22.000000
"# 173 - The Stills, Novillero, Fred Eaglesmith",27.000000
"# 174 - The Organ, Sloan, Jill Barber",27.500000
#1,24.246154
#1 Crush,25.000000


In [47]:
plays["Avg Age"]=age['Age']
plays.head()

,Plays,Pct Males,Avg Age
Track,,,
! (Foreword),8,100.00,22.000000
"# 173 - The Stills, Novillero, Fred Eaglesmith",1,100.00,27.000000
"# 174 - The Organ, Sloan, Jill Barber",2,50.00,27.500000
#1,65,55.38,24.246154
#1 Crush,7,85.71,25.000000


### Percentage of plays that came from a user who played the song exactly once.



In [48]:
data2=data.copy()

In [49]:
#data2=data2.drop(['Age','Gender'],axis=1)
data2.head()

,UserID,Gender,Age,Track
0,user_000001,m,25,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,m,25,Composition 0919 (Live_2009_4_15)
2,user_000001,m,25,Mc2 (Live_2009_4_15)
3,user_000001,m,25,Hibari (Live_2009_4_15)
4,user_000001,m,25,Mc1 (Live_2009_4_15)


In [50]:
pt = data2.pivot_table(index=data2.Track, columns=data2.UserID, aggfunc='count')

In [51]:
pt.head(10)

Age              \
UserID                                         user_000001 user_000002   
Track                                                                    
! (Foreword)                                           NaN         NaN   
# 173 - The Stills, Novillero, Fred Eaglesmith         NaN         NaN   
# 174 - The Organ, Sloan, Jill Barber                  NaN         NaN   
#1                                                     NaN         NaN   
#1 Crush                                               NaN         NaN   
#1 Crush (Original Version)                            NaN         NaN   
#1 Hit Jam                                             NaN         NaN   
#1 Target                                              NaN         NaN   
#102 - Saskatoon Secret                                NaN         NaN   
#105 - The Canadian Dictionary Special                 NaN         NaN   

                                                                        \
UserID                                         user_000003 user_000004   
Track                                                                    
! (Foreword)                                           NaN         NaN   
# 173 - The Stills, Novillero, Fred Eaglesmith         NaN         NaN   
# 174 - The Organ, Sloan, Jill Barber                  NaN         NaN   
#1                                                     NaN         NaN   
#1 Crush                                               NaN         1.0   
#1 Crush (Original Version)                            NaN        10.0   
#1 Hit Jam                                             NaN         NaN   
#1 Target                                              NaN         NaN   
#102 - Saskatoon Secret                                NaN         NaN   
#105 - The Canadian Dictionary Special                 NaN         NaN   

                                                                        \
UserID                                         user_000005 user_000006   
Track                                                                    
! (Foreword)                                           NaN         NaN   
# 173 - The Stills, Novillero, Fred Eaglesmith         NaN         NaN   
# 174 - The Organ, Sloan, Jill Barber                  NaN         NaN   
#1                                                     NaN         NaN   
#1 Crush                                               NaN         NaN   
#1 Crush (Original Version)                            NaN         NaN   
#1 Hit Jam                                             NaN         NaN   
#1 Target                                              NaN         NaN   
#102 - Saskatoon Secret                                NaN         NaN   
#105 - The Canadian Dictionary Special                 NaN         NaN   

                                                                        \
UserID                                         user_000007 user_000008   
Track                                                                    
! (Foreword)                                           NaN         NaN   
# 173 - The Stills, Novillero, Fred Eaglesmith         NaN         NaN   
# 174 - The Organ, Sloan, Jill Barber                  NaN         NaN   
#1                                                     NaN         NaN   
#1 Crush                                               NaN         NaN   
#1 Crush (Original Version)                            NaN         NaN   
#1 Hit Jam                                             NaN         NaN   
#1 Target                                              NaN         NaN   
#102 - Saskatoon Secret                                NaN         NaN   
#105 - The Canadian Dictionary Special                 NaN         NaN   

                                                                        \
UserID                                         user_000009 user_000010   
Track                                                  

In [52]:
pt.fillna(0,inplace=True)   

In [53]:
pt.head()

Age              \
UserID                                         user_000001 user_000002   
Track                                                                    
! (Foreword)                                           0.0         0.0   
# 173 - The Stills, Novillero, Fred Eaglesmith         0.0         0.0   
# 174 - The Organ, Sloan, Jill Barber                  0.0         0.0   
#1                                                     0.0         0.0   
#1 Crush                                               0.0         0.0   

                                                                        \
UserID                                         user_000003 user_000004   
Track                                                                    
! (Foreword)                                           0.0         0.0   
# 173 - The Stills, Novillero, Fred Eaglesmith         0.0         0.0   
# 174 - The Organ, Sloan, Jill Barber                  0.0         0.0   
#1                                                     0.0         0.0   
#1 Crush                                               0.0         1.0   

                                                                        \
UserID                                         user_000005 user_000006   
Track                                                                    
! (Foreword)                                           0.0         0.0   
# 173 - The Stills, Novillero, Fred Eaglesmith         0.0         0.0   
# 174 - The Organ, Sloan, Jill Barber                  0.0         0.0   
#1                                                     0.0         0.0   
#1 Crush                                               0.0         0.0   

                                                                        \
UserID                                         user_000007 user_000008   
Track                                                                    
! (Foreword)                                           0.0         0.0   
# 173 - The Stills, Novillero, Fred Eaglesmith         0.0         0.0   
# 174 - The Organ, Sloan, Jill Barber                  0.0         0.0   
#1                                                     0.0         0.0   
#1 Crush                                               0.0         0.0   

                                                                        \
UserID                                         user_000009 user_000010   
Track                                                                    
! (Foreword)                                           0.0         0.0   
# 173 - The Stills, Novillero, Fred Eaglesmith         0.0         0.0   
# 174 - The Organ, Sloan, Jill Barber                  0.0         0.0   
#1                                                     0.0        23.0   
#1 Crush                                               0.0         0.0   

                                                   ...          Gender  \
UserID                                             ...     user_000024   
Track                                              ...                   
! (Foreword)                                       ...             0.0   
# 173 - The Stills, Novillero, Fred Eaglesmith     ...             0.0   
# 174 - The Organ, Sloan, Jill Barber              ...             0.0   
#1                                                 ...             0.0   
#1 Crush                                           ...             0.0   

                                                                        \
UserID                                         user_000025 user_000026   
Track                                                                    
! (Foreword)                                           0.0         8.0   
# 173 - The Stills, Novillero, Fred Eaglesmith         0.0         0.0   
# 174 - The Organ, Sloan, Jill Barber                  0.0         0.0   
#1                                                   

In [54]:
def pct_one(row):
    total= (row.sum(axis=0))
    return (len(row ==1)/total)

In [55]:
plays["Pct One"] = round(pt.apply(pct_one, axis=1),2)

In [56]:
plays.head()

,Plays,Pct Males,Avg Age,Pct One
Track,,,,
! (Foreword),8,100.00,22.000000,4.12
"# 173 - The Stills, Novillero, Fred Eaglesmith",1,100.00,27.000000,33.00
"# 174 - The Organ, Sloan, Jill Barber",2,50.00,27.500000,16.50
#1,65,55.38,24.246154,0.51
#1 Crush,7,85.71,25.000000,4.71


percentage of plays that came from a user who played the song at least five times  

In [57]:
def pct_five(row):
    total = row.sum(axis=0)
    return (len(row>=5)/total)

In [58]:
plays['Pct Five or more'] = round(pt.apply(pct_five,axis=1),2)

In [59]:
plays.head()

,Plays,Pct Males,Avg Age,Pct One,Pct Five or more
Track,,,,,
! (Foreword),8,100.00,22.000000,4.12,4.12
"# 173 - The Stills, Novillero, Fred Eaglesmith",1,100.00,27.000000,33.00,33.00
"# 174 - The Organ, Sloan, Jill Barber",2,50.00,27.500000,16.50,16.50
#1,65,55.38,24.246154,0.51,0.51
#1 Crush,7,85.71,25.000000,4.71,4.71


## Pickling

In [60]:
pickle_out = open('data.pkl', 'wb')
# Pickle the 'data' dictionary using the highest protocol available.
pickle.dump(plays, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()